Libraries

In [22]:
import numpy as np
from scipy.sparse import coo_matrix
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import AerSimulator

from qiskit.quantum_info import Operator

import matplotlib.pyplot as plt
import networkx as nx

Text to binary convertor

In [1]:
def text_to_binary(text):
    return ' '.join(format(ord(char), '08b') for char in text)

In [4]:
text = "Hi"
binary_string = text_to_binary(text)
print(binary_string)

01001000 01101001


lively DTQW

In [43]:
def G_matrix():
    grover_matrix = np.array([[-1/3, 2/3, 2/3],
                              [2/3, -1/3, 2/3],
                              [2/3, 2/3, -1/3]])
    # grover comes from |a_1|^2 + |a_2|^2 + |a_3|^2 = 1

    return grover_matrix

def create_grover_operator(grover_matrix):
    return Operator(grover_matrix)

def shift_operator(N, tau):
    if N <= 0:
        raise ValueError("N must be a positive integer.")
    
    if tau > np.floor(N / 2):
        raise ValueError("tau must be greater than or equal to N/2.")

    dim = 3 * N
    S = np.zeros((dim, dim), dtype=complex)

    for x in range(N):
        # coin = 0: move to x+1 mod N
        S[( (x + 1) % N ) + 0 * N, x + 0 * N] = 1

        # coin = 1: move to x-1 mod N
        S[( (x - 1) % N ) + 1 * N, x + 1 * N] = 1

        # coin = 2: move to x+tau mod N
        S[( (x + tau) % N ) + 2 * N, x + 2 * N] = 1

    return S
    

def U_matrix(S, C, N):
    if N <= 0:
        raise ValueError("N must be a positive integer.")
    
    identity = np.eye(N)
    U = S @ (np.kron(identity, C))
    return U

In [66]:
N = 2
tau = 1
S = shift_operator(N, tau)
print(S)
G = G_matrix()
#print(G)
U = U_matrix(S, G, N)
print(U)

[[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]]
[[ 0.66666667+0.j -0.33333333+0.j  0.66666667+0.j  0.        +0.j
   0.        +0.j  0.        +0.j]
 [-0.33333333+0.j  0.66666667+0.j  0.66666667+0.j  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j -0.33333333+0.j
   0.66666667+0.j  0.66666667+0.j]
 [ 0.66666667+0.j  0.66666667+0.j -0.33333333+0.j  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.66666667+0.j
   0.66666667+0.j -0.33333333+0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.66666667+0.j
  -0.33333333+0.j  0.66666667+0.j]]


Initial state creation

In [53]:
def initial_state(N, a1, a2, a3):

    a1 = np.full(N, a1, dtype=complex) if np.isscalar(a1) else np.array(a1, dtype=complex)
    a2 = np.full(N, a2, dtype=complex) if np.isscalar(a2) else np.array(a2, dtype=complex)
    a3 = np.full(N, a3, dtype=complex) if np.isscalar(a3) else np.array(a3, dtype=complex)
    
    # Build the full state vector |x> ⊗ |coin>
    state = np.zeros((N*3,), dtype=complex)
    for x in range(N):
        state[x*3 + 0] = a1[x]  # coin state 0
        state[x*3 + 1] = a2[x]  # coin state 1
        state[x*3 + 2] = a3[x]  # coin state 2
    
    # Normalize
    norm = np.linalg.norm(state)
    if norm != 0:
        state /= norm
    
    return state

In [64]:

N = 2
psi0 = initial_state(N, a1=1/np.sqrt(3), a2=1/np.sqrt(3), a3=1/np.sqrt(3))
print("Initial state vector:\n", psi0)
print("Norm =", np.linalg.norm(psi0))

# print the probability distribution just in positional space
def print_probability_distribution(state):
    probabilities = np.abs(state)**2
    for i, prob in enumerate(probabilities):
        print(f"State |{i}>: Probability = {prob:.4f}")
print_probability_distribution(psi0)

psi_1 = np.dot(U, psi0)
print("State vector after applying U:\n", psi_1)
print(print_probability_distribution(psi_1))

Initial state vector:
 [0.40824829+0.j 0.40824829+0.j 0.40824829+0.j 0.40824829+0.j
 0.40824829+0.j 0.40824829+0.j]
Norm = 1.0
State |0>: Probability = 0.1667
State |1>: Probability = 0.1667
State |2>: Probability = 0.1667
State |3>: Probability = 0.1667
State |4>: Probability = 0.1667
State |5>: Probability = 0.1667
State vector after applying U:
 [0.40824829+0.j 0.40824829+0.j 0.40824829+0.j 0.40824829+0.j
 0.40824829+0.j 0.40824829+0.j]
State |0>: Probability = 0.1667
State |1>: Probability = 0.1667
State |2>: Probability = 0.1667
State |3>: Probability = 0.1667
State |4>: Probability = 0.1667
State |5>: Probability = 0.1667
None


In [63]:
def initial_state_single_position(N):
    """
    Initial state: particle at position x=0 with coin state (1,1,1)/sqrt(3)
    """
    state = np.zeros((N * 3,), dtype=complex)

    # Coin superposition at position 0
    coin_superpos = np.array([1, 1, 1], dtype=complex) / np.sqrt(3)
    
    # Fill position x=0 in the vector
    state[0*3 : 0*3 + 3] = coin_superpos
    
    return state

# Example: N=5
N = 2
psi0 = initial_state_single_position(N)
print("Initial state vector:\n", psi0)
print("Norm =", np.linalg.norm(psi0))

Initial state vector:
 [0.57735027+0.j 0.57735027+0.j 0.57735027+0.j 0.        +0.j
 0.        +0.j 0.        +0.j]
Norm = 1.0


Making circuit directly (based on article: 2023 Hou)

In [20]:
def shift_operator(qc, qpos, qcoin, tau):
    qc.x(qcoin[0]); qc.x(qcoin[1])
    increment_mod2n(qc, qpos, controls=qcoin, ctrl_state='11')
    qc.x(qcoin[0]); qc.x(qcoin[1])

    # c=1: |01>，-1 ≡ +(N-1)
    qc.x(qcoin[0])
    add_tau_mod2n(qc, qpos, (N-1), controls=qcoin, ctrl_state='11')
    qc.x(qcoin[0])

    # c=2: |10>，+tau
    qc.x(qcoin[1])  # 把 |10> 變成 |11>
    add_tau_mod2n(qc, qpos, tau, controls=qcoin, ctrl_state='11')
    qc.x(qcoin[1])

def increment_mod2n(qc, qpos, controls=None, ctrl_state=None):
    # 簡單 ripple-carry +1
    n = len(qpos)
    qc.x(qpos[0]) if controls is None else qc.mcx(controls, qpos[0])
    for i in range(1, n):
        if controls is None:
            qc.mcx(qpos[:i], qpos[i])
        else:
            qc.mcx(list(controls)+qpos[:i], qpos[i])

def add_tau_mod2n(qc, qpos, tau, controls=None, ctrl_state=None):
    for i in range(len(qpos)):
        if (tau >> i) & 1:
            if controls is None:
                qc.x(qpos[i])
            else:
                qc.mcx(controls, qpos[i])

Print probability distribution

In [ ]:
def plotBarProb(prob):      ### 化成長條圖
    """
    prob: numpy array, 機率分佈 (長度 N)
    """
    N = len(prob)
    x = np.arange(N)

    fig, ax = plt.subplots(figsize=(6,4))
    bars = ax.bar(x, prob, color='skyblue', edgecolor='black')

    # 標籤與刻度
    ax.set_xlabel("Vertex index")
    ax.set_ylabel("Probability")
    ax.set_title("Quantum walk position probability")
    ax.set_xticks(x)
    ax.set_ylim(0, max(prob)*1.2)

    # 在每個 bar 上加數值標籤
    for bar, p in zip(bars, prob):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f"{p:.3f}", ha='center', va='bottom', fontsize=8)

    plt.show()

In [25]:
N = 8                     # 頂點數（示範用，小N方便畫圖）
n_pos = int(np.log2(N))   # 編碼位置所需 qubit 數
tau_map = {'0': 0, '1': 2}
#msg_bits = '0110'         # 示範的訊息 bits
a1 = a2 = a3 = 1/np.sqrt(3)

msg = 'ayiii' 
msg_bits = ''.join(f"{ord(c):08b}" for c in msg)
print("Message bits:", msg_bits)



#  Grover coin (3D 嵌入 4D)
sv = np.array([1,1,1], dtype=float)/np.sqrt(3)
G3 = 2*np.outer(sv, sv) - np.eye(3)
G4 = np.eye(4)
G4[:3,:3] = G3

print(G4)

Message bits: 0110000101111001011010010110100101101001
[[-0.33333333  0.66666667  0.66666667  0.        ]
 [ 0.66666667 -0.33333333  0.66666667  0.        ]
 [ 0.66666667  0.66666667 -0.33333333  0.        ]
 [ 0.          0.          0.          1.        ]]
